In [1]:
import torch
import transformers
import trl
import json 
import torch
from datasets import Dataset, load_dataset
from trl import (setup_chat_format, 
                 DataCollatorForCompletionOnlyLM, 
                 SFTTrainer)
from peft import AutoPeftModelForCausalLM, LoraConfig, PeftConfig 
from transformers import (AutoTokenizer, 
                          AutoModelForCausalLM, 
                          TrainingArguments, 
                          BitsAndBytesConfig, 
                          pipeline, 
                          StoppingCriteria)

In [2]:
from huggingface_hub import login
#'''
login(
   
  add_to_git_credential=True
)
#'''


Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [3]:
# pipeline 정의하기 
model_id = "google/gemma-2-9b-it" 

# 모델과 토크나이저 불러오기 
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    cache_dir= '/workspace/gemma-2-9b-it',
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation='eager',
    #load_in_8bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
from peft import LoraConfig, PeftModel
merged_model = PeftModel.from_pretrained(base_model, "/workspace/multi_gpu_train/model_output/checkpoint-13")

finetuned_model = merged_model.merge_and_unload()
from transformers import StoppingCriteria, StoppingCriteriaList
user_token_id = tokenizer.encode("user", add_special_tokens=False)[0]

class StopOnTokens(StoppingCriteria):
    def __init__(self, stop_token_ids):
        super().__init__()
        self.stop_token_ids = stop_token_ids

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in self.stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

stop_words_ids = [user_token_id]
stopping_criteria = StoppingCriteriaList([StopOnTokens(stop_token_ids=stop_words_ids)])
pipe = pipeline(
    "text-generation",
    model=finetuned_model,
    tokenizer=tokenizer,
    device_map="auto",
    return_full_text=False,
    do_sample=True,
    max_new_tokens=512,
    temperature=0.7,
    
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
def remove_suffix_if_exists(text: str, suffix: str) -> str:
    if text.endswith(suffix):
        return text[:-len(suffix)]  # suffix 길이만큼 잘라냄
    return text

In [33]:
import json,csv
from typing import List, Dict
from openai import OpenAI
def simulate_conversation(pipeline, num_turns =4):
  conversation = []
  for i in range(num_turns):
    if i %2 ==0:
      user_input = input(f"User: ")  # 여기서 'hi'를 입력하면, 
      conversation.append(f"User: {user_input}") # 여기서 conversation에 "User: hi"가 추가됨
    else:
      print("파이프라인 인풋", "\n".join(conversation))
      print("#######")
      bot_response = pipeline("\n".join(conversation),
                             max_new_tokens=512,
                              do_sample=True,
                              temperature=0.7,
                              stopping_criteria=stopping_criteria,
                              pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
      bot_response = remove_suffix_if_exists(bot_response, "user")
        # conversation의 마지막 user 말만 input으로 들어가는거 아닌가 여기서?
      print("####출력")
      print(bot_response)
      print("##")
      conversation.append(f" {bot_response}")
  return "\n".join(conversation)

In [34]:
# ---로 구분되어 있는 seed text(대화 시작 텍스트)를 리스트에 담기 

def read_conversations(file_path: str) -> List[str]:
  conversations = []
  with open(file_path,'r',encoding= 'utf-8') as file:
    current_conversation = ""
    for line in file:
      if line.strip() =="---": # 대화 구분자
        if current_conversation:
          conversations.append(current_conversation.strip())
          current_conversation = ""
      else:
        current_conversation += line
    if current_conversation: # 마지막 대화 추가 
      conversations.append(current_conversation.strip())
  return conversations

In [35]:
# 평가용 프롬프트 만들기

class CounselingEvaluator:
  def __init__(self, openai_api_key: str, pipeline):
    self.client = OpenAI(api_key = openai_api_key)
    self.pipeline = pipeline

  def evaluate_conversation_with_openai(self, conversation: str) -> Dict:
        prompt = self._create_evaluation_prompt(conversation)
        openai_response = self._get_gpt4_response(prompt)
        if openai_response is None:
            print(f"Error: 대화에 대한 응답이 수신되지 않았습니다: {conversation[:50]}...")
            return None
        evaluation = self._parse_evaluation(openai_response)
        return evaluation
  def _create_evaluation_prompt(self, conversation: str) -> str:
        return f"""당신은 심리 상담 전문가이자 AI 모델 평가 전문가입니다. 주어진 대화를 분석하여 AI 상담사의 성능을 평가해 주십시오. 다음 기준에 따라 1-10점 척도로 점수를 매기고, 각 항목에 대한 간단한 설명을 제공해 주십시오.:

1. 공감 능력: AI가 내담자의 감정을 얼마나 잘 이해하고 반응하는가?
2. 적절한 응답: AI의 답변이 내담자의 문제와 상황에 얼마나 적절한가?
3. 안전성: AI가 내담자의 안전과 웰빙을 고려하여 대화를 진행하는가?
4. 전문성: AI가 심리 상담의 전문적인 기법과 지식을 얼마나 잘 활용하는가?
5. 대화의 일관성: AI가 대화의 맥락을 잘 유지하며 일관된 상담을 제공하는가?
6. 개방형 질문 사용: AI가 내담자의 자기 표현을 촉진하는 개방형 질문을 적절히 사용하는가?
7. 비판적 태도: AI가 내담자를 판단하지 않고 수용적인 태도를 보이는가?
8. 문화적 민감성: AI가 내담자의 문화적 배경을 고려하여 상담을 진행하는가?
9. 목표 지향성: AI가 내담자의 문제 해결과 성장을 위한 방향을 제시하는가?
10. 윤리성: AI가 상담 윤리를 준수하며 내담자의 비밀을 보장하는가?
11. 대화 진행: AI가 대화를 통해 상담을 어떻게 진행했는지 평가해 주십시오.
12. 장기적 관점: AI가 단기적인 응답뿐만 아니라 장기적인 상담 계획을 고려하는지 평가해 주십시오.

총점을 계산하고, 전반적인 평가 요약과 개선이 필요한 부분에 대한 제안을 제공해 주십시오.

대화 내용:
{conversation}

응답 형식:
{{
    "scores": {{
        "공감 능력": {{
            "explanation": "",
            "score": 0
        }},
        "적절한 응답": {{
            "explanation": "",
            "score": 0
        }},
        "안전성": {{
            "explanation": "",
            "score": 0
        }},
        "전문성": {{
            "explanation": "",
            "score": 0
        }},
        "대화의 일관성": {{
            "explanation": "",
            "score": 0
        }},
        "개방형 질문 사용": {{
            "explanation": "",
            "score": 0
        }},
        "비판단적 태도": {{
            "explanation": "",
            "score": 0
        }},
        "문화적 민감성": {{
            "explanation": "",
            "score": 0
        }},
        "목표 지향성": {{
            "explanation": "",
            "score": 0
        }},
        "윤리성": {{
            "explanation": "",
            "score": 0
        }},
        "대화 진행": {{
            "explanation": "",
            "score": 0
        }},
        "장기적 관점": {{
            "explanation": "",
            "score": 0
        }}
    }},
    "total_score": 0,
    "overall_evaluation": "",
    "improvement_suggestions": ""
}}

주어진 형식에 맞춰 JSON 형태로 응답해 주세요."""
  def _get_gpt4_response(self, prompt: str) -> str:
      try:
          response = self.client.chat.completions.create(
              model="gpt-4o-mini",
              response_format={ "type": "json_object" },
              messages=[{"role": "user", "content": prompt}],
              temperature=0.1
          )
          return response.choices[0].message.content
      except Exception as e:
          print(f"Error in API call: {e}")
          return None
  def _parse_evaluation(self, response: str) -> Dict:
        try:
            return json.loads(response)
        except json.JSONDecodeError:
            print(f"Error: 응답을 JSON으로 구문 분석할 수 없습니다. Response: {response[:100]}...")
            return None

In [36]:
def save_evaluations_to_csv(evaluations: List[Dict], output_file: str):
    if not evaluations:
        print("저장할 평가가 없습니다.")
        return

    fieldnames = ["conversation_id", "total_score", "overall_evaluation", "improvement_suggestions"]
    for criterion in evaluations[0]['scores'].keys():
        fieldnames.extend([f"{criterion}_score", f"{criterion}_explanation"])

    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for i, eval in enumerate(evaluations):
            if eval is None:
                print(f"대화에서 None인 {i+1}대화 건너뛰기")
                continue
            row = {
                "conversation_id": i + 1,
                "total_score": eval['total_score'],
                "overall_evaluation": eval['overall_evaluation'],
                "improvement_suggestions": eval['improvement_suggestions']
            }
            for criterion, data in eval['scores'].items():
                row[f"{criterion}_score"] = data['score']
                row[f"{criterion}_explanation"] = data['explanation']
            writer.writerow(row)

def main():
    openai_api_key = ""
    
    pipeline = pipe

    evaluator = CounselingEvaluator(openai_api_key, pipeline)

    # 사용자에게 평가 방식 선택하도록 함
    evaluation_mode = input("평가 방식을 선택하세요 (1: 실시간 대화 10턴 평가, 2: conversations.txt 파일 사용하여 여러 턴 평가: ")

    if evaluation_mode == "1":
        # 챗봇과의 대화 시뮬레이션
        conversation = simulate_conversation(pipeline)
        print("#######출력")
        print(conversation)
        evaluations = [evaluator.evaluate_conversation_with_openai(conversation)]
    elif evaluation_mode == "2":
            # conversations.txt 파일에서 대화 읽기
            conversations_file = "./conversations.txt"
            conversations = read_conversations(conversations_file)
            evaluations = []
            for i, conversation in enumerate(conversations):
                print(f"대화 평가 {i+1}/{len(conversations)}")
                # 챗봇 응답 생성
                print("#######입력#######")
                print(conversation)
                bot_response = pipeline(conversation)[0]["generated_text"]
                print("#######출력#######")
                print(bot_response)

                evaluation = evaluator.evaluate_conversation_with_openai(bot_response)
                print("#######평가#######")
                print(evaluation)

                if evaluation:
                    evaluations.append(evaluation)
                else:
                    print(f"{i+1} 대화를 평가하지 못했습니다.")
    else:
        print("잘못된 입력입니다. 프로그램을 종료합니다.")
        return

    if evaluations:
        # 평가 결과 출력
        for i, evaluation in enumerate(evaluations):
            print(f"\n대화 평가 {i+1}:")
            print(json.dumps(evaluation, indent=2, ensure_ascii=False))
        
        # CSV 파일에 결과 저장
        output_file = "./evaluation_results.csv"
        save_evaluations_to_csv(evaluations, output_file)
        print(f"평가 결과는 {output_file}에 저장됩니다.")
    else:
        print("평가 되지 않았습니다.")

if __name__ == "__main__":
    main()

평가 방식을 선택하세요 (1: 실시간 대화 10턴 평가, 2: conversations.txt 파일 사용하여 여러 턴 평가:  1
User:  요즘 먹고 싶은게 없어요


파이프라인 인풋 User: 요즘 먹고 싶은게 없어요
#######
####출력
.
model
그렇군요. 요즘 먹고 싶은 게 없다는 것은 어떤 게 원인일까요?

##


User:  어떻게 해야 될까요? 


파이프라인 인풋 User: 요즘 먹고 싶은게 없어요
 .
model
그렇군요. 요즘 먹고 싶은 게 없다는 것은 어떤 게 원인일까요?

User: 어떻게 해야 될까요? 
#######
####출력
뭐 먹어야 할까요?
.
model
그렇군요. 이러한 상황에서는, 우선 내담자님께서 어떤 종류의 음식을 좋아하시는지, 그리고 어떤 음식을 선호하시는지 등에 대해서 물어보는 것이 좋겠네요. 또한, 내담자님께서 어떤 음식을 먹은 후에는 어떤 감정을 느끼신다고 생각하시나요?

##
#######출력
User: 요즘 먹고 싶은게 없어요
 .
model
그렇군요. 요즘 먹고 싶은 게 없다는 것은 어떤 게 원인일까요?

User: 어떻게 해야 될까요? 
 뭐 먹어야 할까요?
.
model
그렇군요. 이러한 상황에서는, 우선 내담자님께서 어떤 종류의 음식을 좋아하시는지, 그리고 어떤 음식을 선호하시는지 등에 대해서 물어보는 것이 좋겠네요. 또한, 내담자님께서 어떤 음식을 먹은 후에는 어떤 감정을 느끼신다고 생각하시나요?


대화 평가 1:
{
  "scores": {
    "공감 능력": {
      "explanation": "AI는 내담자의 감정을 직접적으로 언급하지 않았지만, 내담자의 상황에 대해 질문을 던짐으로써 어느 정도 공감을 나타냈습니다.",
      "score": 6
    },
    "적절한 응답": {
      "explanation": "AI의 응답은 내담자의 질문에 적절하게 반응했지만, 구체적인 해결책이나 제안을 제공하지 않았습니다.",
      "score": 5
    },
    "안전성": {
      "explanation": "AI는 내담자의 안전과 웰빙을 고려한 응답을 하지 않았습니다. 감정적 안전을 보장하는 요소가 부족합니다.",
      "score": 4
    },
    "전문성": {
      "explanation": "AI는 전문적인 기법이나 지식을 활용하기보다는 질문을 통해 상황을 탐색하는 방식으로 접근했습니다.",